# Services (hour)

In [254]:
import pandas as pd

## Database connection

In [255]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [256]:
# Load dimensions
time_dimension = pd.read_sql_table("TIME_DIMENSION", OLAP_connection)
courier_dimension = pd.read_sql_table("COURIER_DIMENSION", OLAP_connection)
customer_dimension = pd.read_sql_table("CUSTOMER_DIMENSION", OLAP_connection)
office_dimension = pd.read_sql_table("OFFICE_DIMENSION", OLAP_connection)
service_status_dimension = pd.read_sql_table("SERVICE_STATUS_DIMENSION", OLAP_connection)

In [257]:
# Load fact table data from OLTP
services = pd.read_sql_table("mensajeria_servicio", OLTP_connection)
service_statuses = pd.read_sql_table("mensajeria_estadosservicio", OLTP_connection)

## Transformation

In [258]:
# Process service statuses to get latest status per service
# Convert 'hora' from datetime.time to string to avoid TypeError
service_statuses["hora_str"] = service_statuses["hora"].astype(str)

In [259]:
# Combine 'fecha' and 'hora_str' into a single datetime column
service_statuses["datetime"] = pd.to_datetime(
		service_statuses["fecha"]
) + pd.to_timedelta(service_statuses["hora_str"])

In [260]:
# Sort by 'datetime' and drop duplicates to get the latest status per 'servicio_id'
latest_status = service_statuses.sort_values("datetime").drop_duplicates(
		"servicio_id", keep="last"
)

In [261]:
# Rename columns for merging
latest_status = latest_status.rename(
    columns={
				"id": "status_record_id",
				"estado_id": "status_id",
				"servicio_id": "service_id",
				"origen_id": "origin_office_id",
				"destino_id": "destination_office_id",
    }
)

In [262]:
# Rename columns in services for consistency
services = services.rename(
		columns={
				"id": "service_id",
				"cliente_id": "customer_id",
				"mensajero_id": "courier_id",
				"origen_id": "origin_office_id",
				"destino_id": "destination_office_id",
		}
)

In [263]:
null_couriers = services["courier_id"].isna().sum()
print(f"Number of services with no assigned courier: {null_couriers}")

Number of services with no assigned courier: 727


In [264]:
# Ignore services with no assigned courier
services = services.dropna(subset=["courier_id"])

In [265]:
# Merge the latest status with services
services = services.merge(
		latest_status[["service_id", "status_id"]], on="service_id", how="left"
)

In [266]:
# Print number of services with no status
null_status = services["status_id"].isna().sum()
print(f"Number of services with no status: {null_status}")

Number of services with no status: 0


In [267]:
# Combine 'fecha_solicitud' and 'hora_solicitud' into 'request_time'
services["hora_solicitud_str"] = services["hora_solicitud"].astype(str)
services["request_time"] = pd.to_datetime(
		services["fecha_solicitud"]
) + pd.to_timedelta(services["hora_solicitud_str"])
services["request_time"] = services["request_time"].dt.floor("H")

In [268]:
# Create a mapping from datetime to time_id
# Ensure that 'time_dimension["date"]' is in datetime format
time_dimension["date"] = pd.to_datetime(time_dimension["date"])
time_mapping = dict(zip(time_dimension["date"], time_dimension["time_id"]))

In [269]:
# Map 'request_time' to 'time_id'
services["time_id"] = services["request_time"].map(time_mapping)

In [270]:
# Check for unmapped 'time_id's
unmapped_times = services["time_id"].isna().sum()
if unmapped_times > 0:
		print(f"Warning: {unmapped_times} records have unmapped 'request_time'.")

In [271]:
# Update: Map to single 'office_id' by selecting 'origin_office_id'
services["office_id"] = services["origin_office_id"]

# Drop the now redundant 'origin_office_id' and 'destination_office_id'
services = services.drop(columns=["origin_office_id", "destination_office_id"])

In [272]:
# Select necessary columns for the fact table
service_fact = services[
		[
				"service_id",
				"customer_id",
				"courier_id",
				"status_id",
				"office_id",
				"time_id",
		]
]

In [273]:
# Aggregate to create 'total_services' by grouping
service_fact = (
		service_fact.groupby(["time_id", "customer_id", "courier_id", "office_id"])
		.agg(total_services=pd.NamedAgg(column="service_id", aggfunc="count"))
		.reset_index()
)

In [274]:
service_fact.head(3)

,time_id,customer_id,courier_id,office_id,total_services
0,2400,5,1.0,177,1
1,2400,5,7.0,15,6
2,2400,5,9.0,15,1


In [275]:
# Add service_fact_table_id
service_fact = service_fact.reset_index().rename(
		columns={"index": "service_fact_hour_table_id"}
)

In [276]:
service_fact.head(10)
# service_fact.shape

,service_fact_hour_table_id,time_id,customer_id,courier_id,office_id,total_services
0,0,2400,5,1.0,177,1
1,1,2400,5,7.0,15,6
2,2,2400,5,9.0,15,1
3,3,2460,5,7.0,15,4
4,4,8340,5,7.0,15,2
5,5,8340,5,7.0,177,2
6,6,8340,5,7.0,237,1
7,7,8340,6,3.0,12,2
8,8,8340,6,36.0,12,1
9,9,12120,5,7.0,15,1


In [277]:
null_couriers = service_fact["courier_id"].isna().sum()
print(f"Number of records with 'courier_id' empty: {null_couriers}")

Number of records with 'courier_id' empty: 0


## Load

In [278]:
# Save the fact table to OLAP
service_fact.to_sql(
		"SERVICE_FACT_HOUR_TABLE", OLAP_connection, if_exists="replace", index=False
)

106